<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
'''
!pip install pandas==2.2.2 -qq
!pip install pyxirr -qq
!pip install ta -qq
!pip install yfinance==0.2.59 -qq
'''

'\n!pip install pandas==2.2.2 -qq\n!pip install pyxirr -qq\n!pip install ta -qq\n!pip install yfinance==0.2.59 -qq\n'

In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  #stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2026-02-24 15:58:45
-------------------
qualified stocks: 86
with latest results: 26
still star stocks: 15
-------------------
Initial Investment:  1.33 C
CY Investment:  1.60 C
Reserve:  2.44 L
Current:  1.40 C
-------------------
Today PnL: -1.22 L (-0.86%)
Current PnL: -34.21 L (-21.38%)
CY Booked + Current PnL: -18.53 L (-11.58%)
-------------------
Total profit:  1.34 L
Total loss:  -35.55 L
-------------------
Total Booked + Current PnL: 8.45 L (6.35%)
Total Booked PnL: 42.66 L (32.07%)
Curr Year Booked PnL: 15.68 L (11.18%)
Prev Year Booked PnL: 26.98 L (20.29%)
Est FTT:  2.41 C
Est FTT PnL: 1.01 C (71.9%)
Deployed:  1.33 C
Current:  1.40 C
CAGR/XIRR %: 2.71%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'FTT Amt', 'Current P/L', 'Current', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%',  'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
18,COALINDIA,1.16,5.98,11.04,17.67,17604.0,8992.0,159452.0,478.51,27.69,58.0,L-LC,11.60,185.0,0.51,1.16,28.00,XY25,ATH,MINING
50,MASFIN,-0.86,3.54,17.42,21.57,17672.0,3465.0,101445.0,397.04,-10.57,58.0,H-SC,6.20,167.0,0.20,0.74,48.04,XR,ATH,FINANCE
77,TTKPRESTIG,2.05,-29.72,42.42,0.09,30044.0,-29952.0,70825.0,770.00,64.35,36.0,M-SC,10.43,250.0,-1.00,0.51,2.05,OX40N,NTT,DURABLES
51,MEDANTA,-1.38,-3.99,29.23,24.08,35628.0,-5061.0,121889.0,1486.00,-4.72,52.0,X-SC,9.34,87.0,-0.14,0.88,11.64,XY24,NTT,HEALTHCARE
36,ICICIGI,-2.55,0.85,18.33,19.34,36992.0,1704.0,201813.0,2252.93,-15.18,50.0,X-MC,1.75,66.0,0.05,1.46,16.93,X40,ATH,INSURANCE


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
11,BANDHANBNK,3.97,-13.15,121.55,92.42,294074.0,-36622.0,241937.0,400.0,107.61,76.0,H-SC,6.55,174.0,-0.12,1.76,34.63,XY24,NTT,BANKS
0,5PAISA,2.59,-37.50,80.65,12.90,110129.0,-81944.0,136552.0,593.0,100.72,46.0,H-SC,7.36,170.0,-0.74,0.99,13.54,OX40N,NTT,FINANCE
1,ABB,2.34,11.11,31.01,45.57,90142.0,29069.0,290688.0,7934.0,-28.55,68.0,H-MC,7.70,125.0,0.32,2.11,29.08,AR,NTT,ELECTRICAL
77,TTKPRESTIG,2.05,-29.72,42.42,0.09,30044.0,-29952.0,70825.0,770.0,64.35,36.0,M-SC,10.43,250.0,-1.00,0.51,2.05,OX40N,NTT,DURABLES
65,SIEMENS,1.92,-9.57,38.80,25.51,65293.0,-17815.0,168280.0,4671.5,41.28,65.0,H-LC,2.51,51.0,-0.27,1.22,24.01,AR,ATH,ELECTRICAL


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
33,HCLTECH,-6.10,-13.09,42.49,23.83,89337.0,-31670.0,210254.0,1908.19,-8.60,18.0,X-LC,3.87,8.0,-0.35,1.53,0.88,X40,ATH,IT
72,TATAELXSI,-4.58,-35.17,103.67,32.03,88598.0,-46373.0,85462.0,9161.00,-18.85,26.0,H-SC,9.35,158.0,-0.52,0.62,0.00,OX40N,NTT,IT
25,EASEMYTRIP,-4.16,-51.66,201.37,45.70,172131.0,-91335.0,85480.0,26.40,-11.26,64.0,M-SC,18.83,204.0,-0.53,0.62,40.84,XY24,NTT,TRAVEL
42,INFY,-3.92,-13.17,54.61,34.24,148366.0,-41206.0,271682.0,1972.00,-31.79,17.0,X-LC,6.32,3.0,-0.28,1.97,0.00,X40,NTT,IT
43,IRCTC,-3.91,-25.95,106.53,52.93,213391.0,-70197.0,200311.0,1269.00,-45.22,45.0,H-MC,11.90,119.0,-0.33,1.45,1.71,XY24,BTT,TRAVEL


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
26,FINCABLES,-1.56,2.89,102.07,107.91,155054.0,4265.0,151909.0,1641.55,-7.66,58.0,M-SC,7.93,220.0,0.03,1.10,14.29,OX40N,ATH,CABLES
77,TTKPRESTIG,2.05,-29.72,42.42,0.09,30044.0,-29952.0,70825.0,770.00,64.35,36.0,M-SC,10.43,250.0,-1.00,0.51,2.05,OX40N,NTT,DURABLES
66,SIS,-1.90,-32.47,79.56,21.26,59656.0,-36049.0,74983.0,528.00,1758.03,30.0,H-SC,4.50,168.0,-0.60,0.54,3.37,OX40N,NTT,MISC
40,INDIGOPNTS,-0.27,-32.64,48.48,0.02,57007.0,-56970.0,117589.0,1408.00,79.13,28.0,M-SC,10.83,222.0,-1.00,0.85,2.28,OX40N,NTT,PAINTS
48,KANSAINER,-0.99,-34.03,72.01,13.47,128101.0,-91773.0,177894.0,340.00,-43.31,26.0,H-SC,11.75,162.0,-0.72,1.29,0.00,XY24,NTT,PAINTS


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
82,VOLTAS,-0.01,20.32,20.68,45.2,47709.0,38958.0,230700.0,1856.0,37.8,63.0,X-MC,7.36,79.0,0.82,1.67,28.5,XY25,NTT,AC


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
18,COALINDIA,1.16,5.98,11.04,17.67,17604.0,8992.0,159452.0,478.51,27.69,58.0,L-LC,11.60,185.0,0.51,1.16,28.00,XY25,ATH,MINING
26,FINCABLES,-1.56,2.89,102.07,107.91,155054.0,4265.0,151909.0,1641.55,-7.66,58.0,M-SC,7.93,220.0,0.03,1.10,14.29,OX40N,ATH,CABLES
50,MASFIN,-0.86,3.54,17.42,21.57,17672.0,3465.0,101445.0,397.04,-10.57,58.0,H-SC,6.20,167.0,0.20,0.74,48.04,XR,ATH,FINANCE
1,ABB,2.34,11.11,31.01,45.57,90142.0,29069.0,290688.0,7934.00,-28.55,68.0,H-MC,7.70,125.0,0.32,2.11,29.08,AR,NTT,ELECTRICAL


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
1,ABB,2.34,11.11,31.01,45.57,90142.0,29069.0,290688.0,7934.00,-28.55,68.0,H-MC,7.70,125.0,0.32,2.11,29.08,AR,NTT,ELECTRICAL
50,MASFIN,-0.86,3.54,17.42,21.57,17672.0,3465.0,101445.0,397.04,-10.57,58.0,H-SC,6.20,167.0,0.20,0.74,48.04,XR,ATH,FINANCE
26,FINCABLES,-1.56,2.89,102.07,107.91,155054.0,4265.0,151909.0,1641.55,-7.66,58.0,M-SC,7.93,220.0,0.03,1.10,14.29,OX40N,ATH,CABLES
39,INDIAMART,-0.19,-4.90,117.38,106.72,137674.0,-6047.0,117289.0,4810.62,-57.03,50.0,H-SC,7.96,140.0,-0.04,0.85,17.33,AR,ATH,MISC
85,ZYDUSLIFE,-0.45,-4.69,42.30,35.63,84495.0,-9823.0,199751.0,1286.17,-17.12,50.0,H-MC,5.10,120.0,-0.12,1.45,11.20,AR,ATH,PHARMA


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
42,INFY,-3.92,-13.17,54.61,34.24,148366.0,-41206.0,271682.0,1972.00,-31.79,17.0,X-LC,6.32,3.0,-0.28,1.97,0.00,X40,NTT,IT
33,HCLTECH,-6.10,-13.09,42.49,23.83,89337.0,-31670.0,210254.0,1908.19,-8.60,18.0,X-LC,3.87,8.0,-0.35,1.53,0.88,X40,ATH,IT
8,AWL,-2.01,-37.86,152.96,57.19,309987.0,-123468.0,202659.0,485.00,-64.48,20.0,X-SC,17.90,81.0,-0.40,1.47,0.00,XY24,NTT,FMCG
73,TCS,-3.83,-26.23,67.52,23.58,215483.0,-113490.0,319139.0,4311.59,-36.09,22.0,X-LC,6.99,2.0,-0.53,2.32,0.00,X40,ATH,IT
12,BATAINDIA,-1.64,-48.46,162.31,35.20,107646.0,-62349.0,66321.0,2096.00,-13.07,29.0,X-SC,21.89,91.0,-0.58,0.48,0.00,X40,NTT,FOOTWEAR


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
78,UNITDSPR,0.32,4.42,15.65,20.77,38264.0,10354.0,244498.0,1644.00,-7.40,59.0,X-MC,0.55,64.0,0.27,1.77,12.15,X40N,NTT,BREWERIES
21,DABUR,0.72,1.28,41.88,43.70,105225.0,3186.0,251254.0,735.00,-8.02,55.0,X-MC,1.39,70.0,0.03,1.82,16.53,XY24,BTT,FMCG
36,ICICIGI,-2.55,0.85,18.33,19.34,36992.0,1704.0,201813.0,2252.93,-15.18,50.0,X-MC,1.75,66.0,0.05,1.46,16.93,X40,ATH,INSURANCE
65,SIEMENS,1.92,-9.57,38.80,25.51,65293.0,-17815.0,168280.0,4671.50,41.28,65.0,H-LC,2.51,51.0,-0.27,1.22,24.01,AR,ATH,ELECTRICAL
34,HINDUNILVR,0.56,-8.25,23.89,13.67,56347.0,-21203.0,235860.0,2922.00,-31.88,50.0,X-LC,2.76,9.0,-0.38,1.71,11.35,XY25,NTT,FMCG


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
12,BATAINDIA,-1.64,-48.46,162.31,35.20,107646.0,-62349.0,66321.0,2096.00,-13.07,29.0,X-SC,21.89,91.0,-0.58,0.48,0.0,X40,NTT,FOOTWEAR
8,AWL,-2.01,-37.86,152.96,57.19,309987.0,-123468.0,202659.0,485.00,-64.48,20.0,X-SC,17.90,81.0,-0.40,1.47,0.0,XY24,NTT,FMCG
58,RELAXO,-1.08,-54.29,233.19,52.31,154733.0,-78805.0,66355.0,1176.00,-51.31,37.0,X-SC,14.02,92.0,-0.51,0.48,0.0,X40N,NTT,FOOTWEAR
42,INFY,-3.92,-13.17,54.61,34.24,148366.0,-41206.0,271682.0,1972.00,-31.79,17.0,X-LC,6.32,3.0,-0.28,1.97,0.0,X40,NTT,IT
53,PGHH,-0.01,-13.88,53.28,32.01,97976.0,-29632.0,183888.0,17616.87,-41.76,32.0,X-MC,9.60,57.0,-0.30,1.33,0.0,X40,ATH,FMCG


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
54,QUESS,-1.80,-32.65,112.08,42.84,49071.0,-21224.0,43782.0,424.00,-55.92,42.0,X-SC,17.90,80.0,-0.43,0.32,5.32,XY24,NTT,MISC
12,BATAINDIA,-1.64,-48.46,162.31,35.20,107646.0,-62349.0,66321.0,2096.00,-13.07,29.0,X-SC,21.89,91.0,-0.58,0.48,0.00,X40,NTT,FOOTWEAR
58,RELAXO,-1.08,-54.29,233.19,52.31,154733.0,-78805.0,66355.0,1176.00,-51.31,37.0,X-SC,14.02,92.0,-0.51,0.48,0.00,X40N,NTT,FOOTWEAR
51,MEDANTA,-1.38,-3.99,29.23,24.08,35628.0,-5061.0,121889.0,1486.00,-4.72,52.0,X-SC,9.34,87.0,-0.14,0.88,11.64,XY24,NTT,HEALTHCARE
35,HONAUT,0.13,-22.37,84.62,43.32,106994.0,-36432.0,126440.0,58357.33,-31.88,44.0,X-SC,11.37,90.0,-0.34,0.92,2.66,X40N,ATH,ELECTRICAL


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
44,ITC,-0.58,-14.79,39.72,19.06,115131.0,-50310.0,289856.0,452.00,-53.09,47.0,X-LC,17.76,1.0,-0.44,2.10,4.95,X40,NTT,FMCG
73,TCS,-3.83,-26.23,67.52,23.58,215483.0,-113490.0,319139.0,4311.59,-36.09,22.0,X-LC,6.99,2.0,-0.53,2.32,0.00,X40,ATH,IT
42,INFY,-3.92,-13.17,54.61,34.24,148366.0,-41206.0,271682.0,1972.00,-31.79,17.0,X-LC,6.32,3.0,-0.28,1.97,0.00,X40,NTT,IT
75,TMPV,-0.63,-16.10,58.92,33.33,161055.0,-52454.0,273346.0,600.00,-82.53,57.0,X-LC,3.00,4.0,-0.33,1.98,11.75,XY24,NTT,AUTO
33,HCLTECH,-6.10,-13.09,42.49,23.83,89337.0,-31670.0,210254.0,1908.19,-8.60,18.0,X-LC,3.87,8.0,-0.35,1.53,0.88,X40,ATH,IT


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
41,INDUSINDBK,1.27,-31.24,93.26,32.89,49512.0,-24116.0,53090.0,1800.00,-350.00,57.0,L-MC,11.52,258.0,-0.49,0.39,46.21,XR,NTT,BANKS
67,SONACOMS,-1.23,-8.41,51.86,39.09,48050.0,-8505.0,92654.0,804.02,-27.36,60.0,M-MC,9.58,193.0,-0.18,0.67,31.07,AR,ATH,AUTO
50,MASFIN,-0.86,3.54,17.42,21.57,17672.0,3465.0,101445.0,397.04,-10.57,58.0,H-SC,6.20,167.0,0.20,0.74,48.04,XR,ATH,FINANCE
18,COALINDIA,1.16,5.98,11.04,17.67,17604.0,8992.0,159452.0,478.51,27.69,58.0,L-LC,11.60,185.0,0.51,1.16,28.00,XY25,ATH,MINING
65,SIEMENS,1.92,-9.57,38.80,25.51,65293.0,-17815.0,168280.0,4671.50,41.28,65.0,H-LC,2.51,51.0,-0.27,1.22,24.01,AR,ATH,ELECTRICAL


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
25,EASEMYTRIP,-4.16,-51.66,201.37,45.70,172131.0,-91335.0,85480.0,26.4,-11.26,64.0,M-SC,18.83,204.0,-0.53,0.62,40.84,XY24,NTT,TRAVEL
1,ABB,2.34,11.11,31.01,45.57,90142.0,29069.0,290688.0,7934.0,-28.55,68.0,H-MC,7.70,125.0,0.32,2.11,29.08,AR,NTT,ELECTRICAL
7,ATULAUTO,-1.45,-18.35,78.28,45.57,125996.0,-36176.0,160956.0,844.0,3483.78,52.0,M-SC,4.48,248.0,-0.29,1.17,21.81,XY24,NTT,AUTO
11,BANDHANBNK,3.97,-13.15,121.55,92.42,294074.0,-36622.0,241937.0,400.0,107.61,76.0,H-SC,6.55,174.0,-0.12,1.76,34.63,XY24,NTT,BANKS
65,SIEMENS,1.92,-9.57,38.80,25.51,65293.0,-17815.0,168280.0,4671.5,41.28,65.0,H-LC,2.51,51.0,-0.27,1.22,24.01,AR,ATH,ELECTRICAL


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,20.63
1,25,45.54
2,50,77.45


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    44.99
MC    30.27
LC    24.68
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     26.98
X40      23.77
X40N     14.86
AR        9.08
XY25      8.91
XR        8.42
OX40N     7.09
SR        0.83
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
X-MC    22.79
H-SC    22.15
X-LC    21.29
X-SC    11.49
M-SC    10.61
H-MC     5.01
M-MC     2.08
H-LC     1.22
L-LC     1.16
M-LC     1.01
L-SC     0.74
L-MC     0.39
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,15.69,-14.06,52.63
FINANCE,13.14,-15.32,60.70
IT,11.12,-43.08,114.78
MISC,6.88,-36.99,93.02
ELECTRICAL,6.39,-6.63,45.54
PAINTS,5.04,-32.81,52.54
INSURANCE,4.82,-1.94,36.83
PHARMA,4.12,-3.58,36.19
AUTO,3.82,-18.43,63.59


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3406989.0,21
X40,1443670.0,15
XR,1332393.0,12
AR,1319126.0,10
X40N,1074471.0,10
OX40N,812185.0,10
XY25,388808.0,6
SR,304958.0,2


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3739531.0,24
M-SC,1452004.0,14
X-MC,1369232.0,15
X-LC,1339841.0,12
X-SC,1220647.0,10
H-MC,388028.0,3
L-SC,189457.0,2
M-LC,152798.0,1
M-MC,98653.0,2


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1260533.0      6
           AR         956298.0      5
           XR         856306.0      7
M-SC       XY24       842087.0      6
X-LC       X40        713521.0      6
X-SC       XY24       623384.0      4
X-MC       X40        545660.0      7
           X40N       433589.0      4
X-SC       X40N       412774.0      4
H-SC       OX40N      361436.0      4
M-SC       OX40N      354758.0      5
H-SC       SR         304958.0      2
X-LC       XY24       274369.0      2
           X40N       228108.0      2
H-MC       XY24       213391.0      1
X-MC       XY25       196758.0      2
           XY24       193225.0      2
X-SC       X40        184489.0      2
M-SC       XR         180311.0      2
H-MC       AR         174637.0      2
M-LC       XR         152798.0      1
X-LC       XY25       123843.0      2
L-SC       OX40N       95991.0      1
           XR          93466.0      1
M-SC       AR          74848.0      1
H-LC       AR          65293.0      1
M-MC       XY25        50603.0      1
L-MC       XR          49512.0      1
M-MC       AR          48050.0      1
L-LC       XY25        17604.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 38.0 seconds
